In [148]:
# !pip3 install requests
# !pip3 install python-dotenv
# !pip3 install gspread
# !pip3 install oauth2client
# !pip3 install google-api-python-client

In [149]:
import os
import json
import requests
import gspread
import datetime
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from dotenv import load_dotenv

In [150]:
load_dotenv('.env')

True

# Retrieve Canvas Inbox starred messages (conversations)

In [151]:
# Replace with your Canvas API token
access_token = os.getenv('CANVAS_ACCESS_TOKEN')
base_url = os.getenv('CANVAS_BASE_URL')

# Headers for authentication
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Parameters to control pagination
params = {
    'per_page': 50  # Adjust this number as needed, up to the maximum allowed by the API
}

In [152]:
conversations_response = requests.get(f'{base_url}/conversations', headers=headers, params=params)
conversations = conversations_response.json()
# print(json.dumps(conversations, indent=4))

# List to store all messages with sender names
sender_data = []

# Get sender names from starred conversations and set status to False
for conversation in conversations:
    if conversation["starred"]:
        sender_id = conversation["participants"][0]["id"]
        sender_name = conversation["participants"][0]["name"]
        sender_data.append([sender_id, sender_name])  # Appending [sender_name, False] to sender_data

print("Number of starred conversations: ", len(sender_data))
# print(sender_data)  # Print sender data for testing

Number of starred conversations:  38


# Save result into Google Sheet

In [153]:
def append_checkbox(col, total_rows, creds=None, spreadsheet=None, worksheet=None):
    service = build('sheets', 'v4', credentials=creds)
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet.id, body={
        'requests': [
            {
                'repeatCell': {
                    'cell': {
                        'dataValidation': {
                            'condition': {'type': 'BOOLEAN'},
                            'showCustomUi': True
                        }
                    },
                    'range': {
                        'sheetId': worksheet.id,
                        'startRowIndex': 1,  # Skip the header row
                        'endRowIndex': total_rows + 1,
                        'startColumnIndex': col,  # Column B
                        'endColumnIndex': col + 1
                    },
                    'fields': 'dataValidation'
                }
            }
        ]
    }).execute()

In [154]:
def change_col_width(col, wid, creds=None, worksheet=None, spreadsheet=None):
    service = build('sheets', 'v4', credentials=creds)
    service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet.id, body = {
        'requests': [
            {
                'updateDimensionProperties': {
                    'range': {
                        'sheetId': worksheet.id,
                        'dimension': 'COLUMNS',
                        'startIndex': col,
                        'endIndex': col + 1
                    },
                    'properties': {
                        'pixelSize': wid  # Set the desired width in pixels
                    },
                    'fields': 'pixelSize'
                }
            }
        ]
    }).execute()

In [155]:
def append_data(sender_data, worksheet, spreadsheet, creds):
    # Collect all existing sender data from all other worksheets
    all_existing_senders = set()
    for sheet in spreadsheet.worksheets():
        if sheet.title != worksheet.title:
            existing_data = sheet.get_all_values()
            existing_senders = {row[0] for row in existing_data if row}  # Assuming sender ids are in the first column
            all_existing_senders.update(existing_senders)

    # Write column headers if the sheet is empty
    if worksheet.get_all_values():
        worksheet.append_row(['Sender ID', 'Sender Name'])

    # Write sender data to the worksheet
    num_breaker = 0
    num_added = 0
    total_line = 0
    num_to_add = len(sender_data)
    person_to_assign = 7
    breaker_row = ['******* Finished my part? *******', '']

    for sender in sender_data:
        if str(sender[0]) not in all_existing_senders:
            worksheet.append_row([sender[0], sender[1]])  # Writing sender data
            print("Added", sender[1])
            num_added += 1
            total_line += 1
            # Add the breaker row after every 7 rows
            if (person_to_assign > 0) and (num_added % round(num_to_add / person_to_assign)) == 0:
                worksheet.append_row(breaker_row)
                print("Added breaker row **********")
                num_to_add -= round(num_to_add / person_to_assign)
                person_to_assign -= 1
                num_breaker += 1
                total_line += 1
                num_added = 0 # Reset the counter for breaker row
        else:
            print(f"Skipped {sender[1]} (already exists)")
            num_to_add -= 1

    change_col_width(1, 200, creds, worksheet, spreadsheet)

    worksheet.update_cell(1, 3, 'Replied?')
    if total_line > 1:
        append_checkbox(2, total_line, creds, spreadsheet, worksheet)

    worksheet.update_cell(1, 4, 'Meta-Feedback Message')
    change_col_width(3, 1000, creds, worksheet, spreadsheet)

    print("Total breaker row(s) added:", num_breaker)

In [156]:
# Authenticate with Google Sheets API
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         "https://www.googleapis.com/auth/spreadsheets"
         ]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Define the folder ID where you want to save the Google Sheets file
folder_id = os.getenv('GOOGLE_DRIVE_FOLDER_ID')
# Define file name for the Google Sheets file
file_name = os.getenv('GOOGLE_SHEET_FILE_NAME')

# Get the current date and time precise to the second and format it
today_date = datetime.now().strftime("%Y-%m-%d-%H-%M")

AttributeError: module 'datetime' has no attribute 'now'

In [ ]:
# Check if a file with the same name already exists
file_exists = False
for file in client.list_spreadsheet_files():
    if file['name'] == file_name:
        print("Same-name Google Sheet file already exists in the folder...")
        file_exists = True
        spreadsheet = client.open(file_name)
        break

# If the file does not exist, create a new one and write sender data to it
if not file_exists:
    print(f'Creating a new Google Sheet file named "{file_name}"...')
    spreadsheet = client.create(file_name, folder_id=folder_id)

    # Rename the default worksheet ('Sheet1') with today's date
    worksheet = spreadsheet.sheet1
    worksheet.update_title(today_date)

    append_data(sender_data, worksheet, spreadsheet, creds)

else:
    print(f'Creating a new worksheet in "{file_name}"...')
    # Add a new worksheet with today's date as the title and write sender data to it
    worksheet = spreadsheet.add_worksheet(title=today_date, rows="100", cols="20")  # Create a new worksheet

    append_data(sender_data, worksheet, spreadsheet, creds)


Creating a new Google Sheet file named "Peer Meta-Feedback List"...
Added Kris Wong
Added Peter Yu
Added Eileen Huynh
Added Vivica Tran
Added Alfredo Nieto
Added breaker row **********
Added Lakshitha Anand
Added Haskell Macaraig
Added Jassan Dhami
Added Dominic Quintero
Added Ameen Salim
Added Stephen Donecker
Added breaker row **********
Added Jacky Liu
Added Subhan Baig
Added Sahil Khatri
Added Henry Wong
Added Haosen Cao
Added breaker row **********
Added Davin Nguyen
Added Keith Low
Added Yogesh Chaudhary
Added Anuj Bhandari
Added Yinghui Huang
Added Armin Irvije
Added breaker row **********
Added Sean Mirahsani
Added William Bradford
Added Luis Bravo Bonilla
Added Priyal Patel
Added Foram Shah
Added breaker row **********
Added Foram Shah
Added Ryan Yu
Added Ronit Amar Bhatia
Added Pavittar Singh
Added Ayse Erel
Added Maggie Chen
Added breaker row **********
Added Ian O'Connell
Added James Xu
Added Brendan Lin
Added Jason Ma
Added Williyam Yared
Added breaker row **********
Total

There is a bug with Google Drive. Even though I have deleted the Google Sheet file from the Google Drive UI, it still shows up in this programming interface. So, I have to delete them using the programming interface to ensure that the file is deleted.

In [ ]:
response = client.list_spreadsheet_files()

# Delete each spreadsheet file
for file in response:
    print(f"Deleting file: {file['name']} ({file['id']})")
    client.del_spreadsheet(file['id'])

Deleting file: Peer Meta-Feedback List (1DgtfclfAeeBtJpioXctUCnm_ekXDP96BSe3tCUZa8-M)
